## Generate single cell heatmaps

In [1]:
# Temporary install of complex heatmap for M1 incompatibilties
# To change later
if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")

BiocManager::install("ComplexHeatmap")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.16 (BiocManager 1.30.19), R 4.2.2 (2022-10-31)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'ComplexHeatmap'”
Old packages: 'curl', 'dplyr', 'gert', 'ggrepel', 'sourcetools'



In [2]:
suppressPackageStartupMessages(library(ComplexHeatmap))
suppressPackageStartupMessages(library(dplyr))

In [3]:
# Set paths and constants
plate_to_focus <- "localhost220513100001_KK22-05-198_FactinAdjusted"
input_data_dir <- file.path("..", "..", "..", "3.process-cfret-features", "data")

output_figure_dir <- "figures"
cp_heatmap_file_noext <- file.path(output_figure_dir, "cp_complex_heatmap")

In [4]:
treatment_colors = c(
    "Null" = "#785EF0",
    "WT" = "#DC267F"
)

## Create a heatmap for CP features

In [5]:
# Load data
cp_file <- file.path(
    input_data_dir, paste0(plate_to_focus, "_sc_norm_fs_cellprofiler.csv.gz")
)

cp_df <- readr::read_csv(
    cp_file,
    col_types = readr::cols(
        .default="d",
        "Metadata_WellRow" = "c",
        "Metadata_WellCol" = "c",
        "Metadata_heart_number" = "c",
        "Metadata_treatment" = "c",
        "Metadata_dose" = "c",
        "Metadata_ImageNumber" = "c",
        "Metadata_Plate" = "c",
        "Metadata_Well" = "c"
    )
)

cell_count_df <- cp_df %>%
    dplyr::group_by(Metadata_Well) %>%
    dplyr::count()

cp_df <- cp_df %>%
    dplyr::left_join(cell_count_df, by = "Metadata_Well") %>%
    dplyr::rename(Metadata_cell_count_per_well = n)

print(dim(cp_df))
head(cp_df, 3)

[1] 17352   629


Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,⋯,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_ER_3_03_256,Nuclei_Texture_SumEntropy_Golgi_3_01_256,Nuclei_Texture_SumEntropy_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_Actin_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Golgi_3_01_256,Nuclei_Texture_SumVariance_Hoechst_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Metadata_cell_count_per_well
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
A,9,9,drug_x,5uM,1,localhost220513100001,A09,1,3,⋯,-0.7714563,0.3830138,1.5279864,1.6033489,-0.4200762,-0.2362416,0.20380578,-0.3547300,0.16163561,342
A,9,9,drug_x,5uM,1,localhost220513100001,A09,2,4,⋯,0.9654590,-0.9223457,-0.4915354,-1.0004204,-0.4608149,-0.4684389,-0.20632511,-0.3310333,-0.18902928,342
A,9,9,drug_x,5uM,1,localhost220513100001,A09,3,7,⋯,-1.4119451,0.1231898,0.9340863,0.9589492,-0.4689575,-0.2375866,-0.03696811,-0.1078189,-0.06320025,342


In [6]:
# Split metadata and feature data
cp_metadata_df <- cp_df %>% dplyr::select(tidyr::starts_with("Metadata"))
cp_meta_cols <- colnames(cp_metadata_df)
cp_df <- cp_df %>% dplyr::select(-!!cp_meta_cols)

In [7]:
# Calculate correlation matrix from feature data
cp_cor_matrix <- t(cp_df) %>% cor()

print(dim(cp_cor_matrix))
head(cp_cor_matrix, 3)

[1] 17352 17352


1.0000000,0.2082696,0.2550026,0.65792044,0.005706803,0.1126529,0.1487917,-0.17203697,0.1974713,0.02974855,⋯,-0.1541250,0.03889222,-0.202090019,NA,-0.08903938,0.12714691,0.01744927,-0.28755110,-0.03750908,-0.1293498
0.2082696,1.0000000,0.1128783,0.07584488,0.454565714,0.2936307,0.3294942,0.34912475,0.1942269,0.36667699,⋯,-0.1389933,-0.13974896,-0.125938834,NA,-0.15653098,-0.03039804,-0.12074641,-0.07961257,-0.19487386,-0.1765305
0.2550026,0.1128783,1.0000000,0.38482358,0.030886270,0.3553611,0.1013867,0.07427077,0.2599918,0.01587037,⋯,-0.1747167,-0.17856485,0.002924819,NA,-0.21859835,0.11813997,0.05696618,-0.06770107,-0.01817249,0.0736789


In [ ]:
ht <- Heatmap(
    cp_cor_matrix,
    name = "Pearson\nCorrelation",
    column_dend_side = "top",
    
    clustering_method_columns = "average",
    clustering_method_rows = "average",
    
    top_annotation = HeatmapAnnotation(
        Treatment = cp_metadata_df$Metadata_treatment,
        CellCount = anno_barplot(
            cp_metadata_df$Metadata_cell_count_per_well,
            height = unit(1, "cm")
        ),
        Well = cp_metadata_df$Metadata_Well,
        Dose = cp_metadata_df$Metadata_dose
    )
)

draw(ht)

In [ ]:
# Save heatmap to file
pdf(paste0(dp_heatmap_file_noext, ".pdf"))
draw(ht)
dev.off()

png(paste0(dp_heatmap_file_noext, ".png"), width = 6.5, height = 6, units = "in", res = 500)
draw(ht)
dev.off()